# End to end learning with closed form solution

**2-stage Method vs. e2e with fair constraints vs. fair objectives**

1. Create a dataset

2. Optimization model

4. Training
    - with different loss functions

5. Evaluation

In [19]:
import cvxpy as cp
import numpy as np
import time
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import torch
import torch.nn as nn
import torch.optim as optim
import time
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

warnings.filterwarnings("ignore")
np.printoptions(suppress=True)
InteractiveShell.ast_node_interactivity = "all"
sys.path.insert(1,'E:\\User\\Stevens\\Spring 2024\\PTO - Fairness\\myUtils')

from torch.utils.data import DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

# Generate synthetic data
from genData import genData
from optDataset import AlphaFairOptDataset
from optModel import optCvModel


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [21]:
def regret_loss(predmodel, data, device):
    """
    A function to calculate regret loss.

    Args:
        predmodel (nn.Module): a regression neural network for cost prediction
        data (tuple): a batch of data from the dataloader
        device (torch.device): device to perform computation on

    Returns:
        torch.Tensor: calculated regret loss
    """
    x, _, _, r_true = data
    x = x.float().to(device)
    r_true = r_true.float().to(device)

    # Predict r_hat
    r_hat = predmodel(x)

    total_regret = torch.tensor(0.0, device=device)
    total_opt_value = torch.tensor(0.0, device=device)

    for i in range(x.size(0)):
        optmodel = optCvModel(
            n=a.shape[1],
            alpha=0.5,
            Q=Q,
            epsilon=epsilon,
            a=a[i],
            r=r[i],
            b=b[i],
            c=c[i]
        )
        # Predicted values
        optmodel.setObj(a[i], r_hat[i].detach().cpu().numpy(), b[i], c[i])
        _, _, opt_value_pred = optmodel.solveP()
        
        # True values
        optmodel.setObj(a[i], r_true[i].cpu().numpy(), b[i], c[i])
        _, _, opt_value_true = optmodel.solve()
        
        # Calculate regret
        total_regret += torch.abs(torch.tensor(opt_value_true - opt_value_pred, device=device))
        total_opt_value += torch.abs(torch.tensor(opt_value_true, device=device))
    
    regret = total_regret / (total_opt_value + 1e-7)
    return regret

In [22]:
num_data = 1000
num_features = 10
num_items = 5
a, b, c, r, x, Q, epsilon = genData(num_data, num_features, num_items)

# Create dataset
dataset = AlphaFairOptDataset(a, b, c, r, x, Q, alpha=0.5, epsilon=epsilon)

# Train-test split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Data loaders
batch_size = 32
loader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define a simple logistic regression model
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, num_items)

    def forward(self, x):
        return self.linear(x)

# Initialize model and optimizer
input_dim = num_features
model = LogisticRegressionModel(input_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [23]:
num_epochs = 10
train_losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for data in loader_train:
        optimizer.zero_grad()
        
        # Calculate regret loss
        # loss = regret_loss(model, data, device)
        loss = criterion(r_hat, r_true)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    train_losses.append(epoch_loss / len(loader_train))
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(loader_train):.4f}')

# Plot the loss curve
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve using Regret')
plt.show()


LogisticRegressionModel(
  (linear): Linear(in_features=10, out_features=5, bias=True)
)

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

The error element 0 of tensors does not require grad and does not have a grad_fn indicates that the tensor you're trying to backpropagate through does not have requires_grad=True. This typically happens when a tensor is created or manipulated in such a way that it loses its gradient tracking.

In our scenario, the issue arises because the regret calculation involves operations that do not maintain gradient tracking. Specifically, the values passed to optmodel.setObj and the calculations within the optCvModel do not support gradient tracking since they involve numpy operations and optimizations that aren't differentiable.

To address this, we need to ensure that our regret calculation is compatible with PyTorch's autograd system. Since the optimization problem involves non-differentiable operations, we should look into alternative methods such as differentiable surrogates or reinforcement learning approaches.

However, a practical way to move forward within the scope of this example is to manually compute gradients using a surrogate loss function that is differentiable. Here, we simplify the regret loss to focus on the PyTorch components.

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# Generate synthetic data for a single sample
num_data = 1
num_features = 10
num_items = 5
a, b, c, r, x, Q, epsilon = genData(num_data, num_features, num_items)

# Create dataset for a single sample
single_sample_dataset = AlphaFairOptDataset(a, b, c, r, x, Q, alpha=0.5, epsilon=epsilon)
single_sample_loader = DataLoader(single_sample_dataset, batch_size=1, shuffle=False)

# Define a simple logistic regression model
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, num_items)

    def forward(self, x):
        return self.linear(x)

# Initialize model and transfer to device
input_dim = num_features
model = LogisticRegressionModel(input_dim).to(device)

# Function to manually calculate regret for a single sample
def manual_regret_calculation(optmodel, a, b, c, r, Q, epsilon, r_hat):
    optmodel.setObj(a, r_hat, b, c)
    _, _, opt_value_pred = optmodel.solveP()
    optmodel.setObj(a, r, b, c)
    _, _, opt_value_true = optmodel.solve()
    total_regret = abs(opt_value_true - opt_value_pred)
    total_opt_value = abs(opt_value_true)
    return total_regret / (total_opt_value + 1e-7)

# Testing the regret function with one-sample test case
def test_regret_function():
    model.eval()
    total_regret = 0.0
    total_opt_value = 0.0

    for data in single_sample_loader:
        x, _, _, _ = data
        x = x.float().to(device)
        with torch.no_grad():
            r_hat = model(x).cpu().numpy()
        
        for i in range(len(x)):
            optmodel = optCvModel(
                n=a.shape[1],
                alpha=0.5,
                Q=Q,
                epsilon=epsilon,
                a=a[i],
                r=r[i],
                b=b[i],
                c=c[i]
            )
            # Manual regret calculation
            manual_regret = manual_regret_calculation(optmodel, a[i], b[i], c[i], r[i], Q, epsilon, r_hat[i])
            
            # Function-based regret calculation
            optmodel.setObj(a[i], r_hat[i], b[i], c[i])
            _, _, opt_value_pred = optmodel.solveP()
            optmodel.setObj(a[i], r[i], b[i], c[i])
            _, _, opt_value_true = optmodel.solve()
            func_regret = abs(opt_value_true - opt_value_pred) / (abs(opt_value_true) + 1e-7)

            print(f'Manual Regret: {manual_regret}')
            print(f'Function-based Regret: {func_regret}')

            assert np.isclose(manual_regret, func_regret), "Regret calculation mismatch!"

# Generate synthetic weights for the model (this would normally come from training)
with torch.no_grad():
    model.linear.weight = nn.Parameter(torch.randn(num_items, num_features).to(device))
    model.linear.bias = nn.Parameter(torch.randn(num_items).to(device))

# Test the regret function
test_regret_function()


Manual Regret: 0.06345776751992198
Function-based Regret: 0.06345776751992198
